In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
270,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241]
271,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242]
272,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243]
273,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244]


In [10]:
df_2020 = df[['Title']]

In [11]:
df_2020

,Title
0,The Grudge
1,Underwater
2,Like a Boss
3,Three Christs
4,Inherit the Viper
...,...
270,We Can Be Heroes
271,News of the World
272,One Night in Miami...
273,Promising Young Woman


In [12]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
df_2020[df_2020['Title'].isnull()]

,Title


In [14]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [15]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [16]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [17]:
df_2020['id'] = df_2020['Title'].map(lambda x: ids(str(x)))

<ipython-input-17-a06ca3f56d32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['id'] = df_2020['Title'].map(lambda x: ids(str(x)))


In [18]:
df_2020

,Title,id
0,The Grudge,1970.0
1,Underwater,443791.0
2,Like a Boss,526019.0
3,Three Christs,418533.0
4,Inherit the Viper,634904.0
...,...,...
270,We Can Be Heroes,615677.0
271,News of the World,581032.0
272,One Night in Miami...,661914.0
273,Promising Young Woman,582014.0


In [19]:
df_2020[df_2020['id'].isnull()]

,Title,id
54,No films had been released on this date as res...,NaN
126,The SpongeBob Movie: Sponge on the Run (Canadi...,NaN


In [20]:
df_2020= df_2020.drop([54,126])

In [21]:
df_2020['id'] = df_2020['id'].astype('int')

In [22]:
def description(movie_id):
        description =""
    
        movie_details = tmdb_movie.details(movie_id)
        if movie_details['overview'] and movie_details['tagline']:
          return movie_details['overview'] + "@#$" + movie_details['tagline']
        elif movie_details['overview'] and not movie_details['tagline']:
          return movie_details['overview'] + "@#$" + ""
        elif not movie_details['overview'] and movie_details['tagline']:
          return ""+ "@#$" + movie_details['tagline']
        else:
          return "@#$"

In [23]:
df_2020['description'] = df_2020['id'].map(lambda x: description(int(x)))

In [24]:
df_2020

,Title,id,description
0,The Grudge,1970,An American nurse living and working in Tokyo ...
1,Underwater,443791,After an earthquake destroys their underwater ...
2,Like a Boss,526019,Two female friends with very different ideals ...
3,Three Christs,418533,Dr. Alan Stone breaks new ground for treatment...
4,Inherit the Viper,634904,"Since the death of their father, the Riley sib..."
...,...,...,...
270,We Can Be Heroes,615677,When alien invaders capture Earth's superheroe...
271,News of the World,581032,A Texan traveling across the wild West bringin...
272,One Night in Miami...,661914,In the aftermath of Cassius Clay's defeat of S...
273,Promising Young Woman,582014,"A young woman, traumatized by a tragic event i..."


In [25]:
df_2020.loc[df_2020['description'] == "@#$"]

,Title,id,description


In [26]:
df_new = df_2020[df_2020['description'].notnull()]

In [27]:
df_new

,Title,id,description
0,The Grudge,1970,An American nurse living and working in Tokyo ...
1,Underwater,443791,After an earthquake destroys their underwater ...
2,Like a Boss,526019,Two female friends with very different ideals ...
3,Three Christs,418533,Dr. Alan Stone breaks new ground for treatment...
4,Inherit the Viper,634904,"Since the death of their father, the Riley sib..."
...,...,...,...
270,We Can Be Heroes,615677,When alien invaders capture Earth's superheroe...
271,News of the World,581032,A Texan traveling across the wild West bringin...
272,One Night in Miami...,661914,In the aftermath of Cassius Clay's defeat of S...
273,Promising Young Woman,582014,"A young woman, traumatized by a tragic event i..."


In [28]:
df_new['description'] = df_new['description'].apply(lambda x: x.split("@#$"))

In [29]:
df_2020=df_new

In [30]:
df_2020['overview'] = df_2020['description'].map(lambda x: x[0])
df_2020['tagline'] = df_2020['description'].map(lambda x: x[1])

In [31]:
df_2020

,Title,id,description,overview,tagline
0,The Grudge,1970,[An American nurse living and working in Tokyo...,An American nurse living and working in Tokyo ...,It never forgives. It never forgets.
1,Underwater,443791,[After an earthquake destroys their underwater...,After an earthquake destroys their underwater ...,7 miles below the ocean surface something has ...
2,Like a Boss,526019,[Two female friends with very different ideals...,Two female friends with very different ideals ...,The world of beauty is about to get ugly.
3,Three Christs,418533,[Dr. Alan Stone breaks new ground for treatmen...,Dr. Alan Stone breaks new ground for treatment...,
4,Inherit the Viper,634904,"[Since the death of their father, the Riley si...","Since the death of their father, the Riley sib...",Some debts can't be paid.
...,...,...,...,...,...
270,We Can Be Heroes,615677,[When alien invaders capture Earth's superhero...,When alien invaders capture Earth's superheroe...,Power comes in all sizes.
271,News of the World,581032,[A Texan traveling across the wild West bringi...,A Texan traveling across the wild West bringin...,Find where you belong.
272,One Night in Miami...,661914,[In the aftermath of Cassius Clay's defeat of ...,In the aftermath of Cassius Clay's defeat of S...,Four legends. One legendary night.
273,Promising Young Woman,582014,"[A young woman, traumatized by a tragic event ...","A young woman, traumatized by a tragic event i...",Revenge never looked so promising.


In [32]:
df_2020['comb'] = df_2020['overview'] + df_2020['tagline']
df_2020['comb'] = df_2020['comb'].fillna('')

In [33]:
df_2020 = df_2020.rename(columns={'Title':'title'})

In [34]:
df_2020 = df_2020.loc[:,['id','overview','tagline','title','comb']]

In [35]:
df_2020

,id,overview,tagline,title,comb
0,1970,An American nurse living and working in Tokyo ...,It never forgives. It never forgets.,The Grudge,An American nurse living and working in Tokyo ...
1,443791,After an earthquake destroys their underwater ...,7 miles below the ocean surface something has ...,Underwater,After an earthquake destroys their underwater ...
2,526019,Two female friends with very different ideals ...,The world of beauty is about to get ugly.,Like a Boss,Two female friends with very different ideals ...
3,418533,Dr. Alan Stone breaks new ground for treatment...,,Three Christs,Dr. Alan Stone breaks new ground for treatment...
4,634904,"Since the death of their father, the Riley sib...",Some debts can't be paid.,Inherit the Viper,"Since the death of their father, the Riley sib..."
...,...,...,...,...,...
270,615677,When alien invaders capture Earth's superheroe...,Power comes in all sizes.,We Can Be Heroes,When alien invaders capture Earth's superheroe...
271,581032,A Texan traveling across the wild West bringin...,Find where you belong.,News of the World,A Texan traveling across the wild West bringin...
272,661914,In the aftermath of Cassius Clay's defeat of S...,Four legends. One legendary night.,One Night in Miami...,In the aftermath of Cassius Clay's defeat of S...
273,582014,"A young woman, traumatized by a tragic event i...",Revenge never looked so promising.,Promising Young Woman,"A young woman, traumatized by a tragic event i..."


In [36]:
df_2020.to_csv('desc2020.csv',index=False)

In [37]:
from google.colab import files
files.download("desc2020.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>